In [1]:
import ee
import geemap
ee.Initialize()
Map = geemap.Map()


geometry=ee.Geometry.Polygon([[[28.723754882812496,41.08814965098298],
                     [28.7841796875,41.08814965098298],
                     [28.7841796875,41.13083153028591],
                    [28.723754882812496,41.13083153028591],
                    [28.723754882812496,41.08814965098298]  
                    ]])


image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(geometry) \
    .filterDate('2021-01-01', '2021-04-27') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B1','B2','B3','B4','B5','B6','B7')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}


Map.addLayer(image, vis_params, "Landsat-8")




def Kmean_Kume( kume_sayisi = 5):


    test_train_noktalari = image.sample(**{
        'region':image.geometry(),
        'scale': 10,
        'numPixels': 40000,
        'geometries': True
    })

   

    
    kumeler = ee.Clusterer.wekaKMeans(kume_sayisi).train(test_train_noktalari)
    
    cozum1 = image.cluster(kumeler, "kMean")
    
    Sınıf_Degerleri = [0, 1, 2, 3, 4]
    Sınıf_Renkleri = ['#CDC0B0', '#00008B', '#838B8B', '327C36', '1FBCD2']
    
    cozum2 = cozum1.set('kMean_sınıf_degerleri', Sınıf_Degerleri)
    cozum2 = cozum2.set('kMean_sınıf_renk_paleti', Sınıf_Renkleri)
    cozum2.remap([0, 1, 2, 3, 4], [1, 2, 3, 4, 5], 1, 'kMean')
    Map.addLayer(cozum2.randomVisualizer(), {}, 'kumeler')

    
    
Kmean_Kume()    

In [2]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…